In [19]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import fit_library
sys.path.append("../PETALO_simulation/SimLib")
import config_sim as CFG
import csv
%matplotlib nbagg
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
config_file = ["R1","R2","R3"]
data_path = "/home/viherbos/DAQ_DATA/NEUTRINOS/CONT_RING/"

df2=pd.DataFrame(data=np.zeros((len(config_file),18),dtype=float),columns=['ACTIVITY','n_ASICS',
                                                           'TE_AS','TE_L1',
                                                           'L1_PROCrate','L1_OUTrate',
                                                           'FIFOA','BUFFER','FIFOB',
                                                           'lost_PROD','lost_CH',
                                                           'lost_ASICout','lost_L1B',
                                                           'WC_CH_FIFO','WC_OLINK_FIFO',
                                                           'WC_L1_A_FIFO','WC_L1_B_FIFO','WC_LATENCY'])


data_file = open(data_path + 'infinity_report.csv','a')
DATA_a = np.array([]).reshape(0,18)

for i in config_file:

    config_file2 = data_path + i + ".json"
    CG = CFG.SIM_DATA(filename = config_file2,read = True)
    CG = CG.data
    filename = CG['ENVIRONMENT']['out_file_name']+"_"+ i + ".h5"
    filename = data_path + filename

    logA         = np.array(pd.read_hdf(filename,key='logA'))
    logB         = np.array(pd.read_hdf(filename,key='logB'))
    log_channels = np.array(pd.read_hdf(filename,key='log_channels'))
    log_outlink  = np.array(pd.read_hdf(filename,key='log_outlink'))
    in_time  = np.array(pd.read_hdf(filename,key='in_time'))
    out_time = np.array(pd.read_hdf(filename,key='out_time'))
    lost         = pd.read_hdf(filename,key='lost')

    out = {'L1': {'in_time': in_time, 'out_time': out_time,
                     'lostL1b': lost['L1b'], 'logA': logA, 'logB': logB},
              'ASICS':{ 'lost_producers':lost['producers'],
                        'lost_channels':lost['channels'],
                        'lost_outlink':lost['outlink'],
                        'log_channels':log_channels,
                        'log_outlink':log_outlink}
             }

    print ("LOST DATA PRODUCER -> CH      = %d" % (out['ASICS']['lost_producers'].sum()))
    print ("LOST DATA CHANNELS -> OUTLINK = %d" % (out['ASICS']['lost_channels'].sum()))
    print ("LOST DATA OUTLINK  -> L1      = %d" % (out['ASICS']['lost_outlink'].sum()))
    print ("LOST DATA L1A -> L1B          = %d" % (np.array(out['L1']['lostL1b']).sum()))

    latency = np.array(out['L1']['out_time'])-np.array(out['L1']['in_time'])

    WC_CH_FIFO    = float(max(out['ASICS']['log_channels'][:,0])/CG['TOFPET']['IN_FIFO_depth'])*100
    WC_OLINK_FIFO = float(max(out['ASICS']['log_outlink'][:,0])/CG['TOFPET']['OUT_FIFO_depth'])*100
    WC_L1_A_FIFO  = float(max(out['L1']['logA'][:,0])/CG['L1']['FIFO_L1a_depth'])*100
    WC_L1_B_FIFO  = float(max(out['L1']['logB'][:,0])/CG['L1']['FIFO_L1b_depth'])*100

        
    DATA_n = np.array([CG['ENVIRONMENT']['ch_rate'],CG['L1']['n_asics'],
                       CG['TOFPET']['TE'],CG['L1']['TE'],
                       CG['L1']['frame_process'],CG['L1']['L1_outrate'],
                       CG['L1']['FIFO_L1a_depth'],CG['L1']['buffer_size'],CG['L1']['FIFO_L1b_depth'],
                       out['ASICS']['lost_producers'].sum(),out['ASICS']['lost_channels'].sum(),
                       out['ASICS']['lost_outlink'].sum(),np.array(out['L1']['lostL1b']).sum(),
                       WC_CH_FIFO,WC_OLINK_FIFO,WC_L1_A_FIFO,WC_L1_B_FIFO,max(latency)])
    
    DATA_a = np.vstack([DATA_a,DATA_n])


df2.loc[:,:] = DATA_a[:,:]
df2.loc[0:0,:].to_csv(data_path + 'infinity_report.csv',header=True,mode='a',float_format='%.4e')
df2.loc[1:,:].to_csv(data_path + 'infinity_report.csv',header=False,mode='a',float_format='%.4e')    
  

{u'ENVIRONMENT': {u'path_to_files': u'/home/viherbos/DAQ_DATA/NEUTRINOS/CONT_RING/', u'n_files': 5, u'file_name': u'p_FR_infinity_', u'out_file_name': u'daq_output', u'n_events': 300, u'ch_rate': 1500000.0, u'temperature': 300}, u'SIPM': {u'size': [1, 3, 3]}, u'TOPOLOGY': {u'radius_ext': 1000, u'n_rows': 16, u'sipm_ext_row': 232, u'radius_int': 1000, u'sipm_int_row': 178}, u'TOFPET': {u'TGAIN': 1, u'TE': 1, u'outlink_rate': 16250000.0, u'OUT_FIFO_depth': 256, u'n_channels': 64, u'IN_FIFO_depth': 4, u'MAX_WILKINSON_LATENCY': 5120}, u'L1': {u'frame_process': 10000000.0, u'L1_outrate': 700000000.0, u'n_asics': 12, u'n_L1': 10000, u'FIFO_L1b_depth': 256, u'FIFO_L1a_depth': 480, u'buffer_size': 256, u'TE': 4}}
LOST DATA PRODUCER -> CH      = 0
LOST DATA CHANNELS -> OUTLINK = 0
LOST DATA OUTLINK  -> L1      = 0
LOST DATA L1A -> L1B          = 0
{u'ENVIRONMENT': {u'path_to_files': u'/home/viherbos/DAQ_DATA/NEUTRINOS/CONT_RING/', u'n_files': 5, u'file_name': u'p_FR_infinity_', u'out_file_name'

In [ ]:
  
#    fit = fit_library.gauss_fit()
#    fig = plt.figure(figsize=(16,8))
#    fit(out['ASICS']['log_channels'][:,0],CG['TOFPET']['IN_FIFO_depth'])
#    fit.plot(axis = fig.add_subplot(231),
#            title = "ASICS Channel Input analog FIFO (4)",
#            xlabel = "FIFO Occupancy",
#            ylabel = "Hits",
#            res = False, fit = False)
#    fig.add_subplot(231).set_yscale('log')
#    fig.add_subplot(231).text(0.4,0.9,(("ASIC Input FIFO reached %.1f %%" % \
#                                            (WC_CH_FIFO))),
#                                            fontsize=8,
#                                            verticalalignment='top',
#                                            horizontalalignment='left',
#                                            transform=fig.add_subplot(231).transAxes)
#    fit(out['ASICS']['log_outlink'][:,0],CG['TOFPET']['OUT_FIFO_depth'])
#    fit.plot(axis = fig.add_subplot(232),
#            title = "ASICS Channels -> Outlink",
#            xlabel = "FIFO Occupancy",
#            ylabel = "Hits",
#            res = False, fit = False)
#    fig.add_subplot(232).set_yscale('log')
#    fig.add_subplot(232).text(0.4,0.9,(("ASIC Outlink FIFO reached %.1f %%" % \
#                                            (WC_OLINK_FIFO))),
#                                            fontsize=8,
#                                            verticalalignment='top',
#                                            horizontalalignment='left',
#                                            transform=fig.add_subplot(232).transAxes)
#    fit(out['L1']['logA'][:,0],CG['L1']['FIFO_L1a_depth'])
#    fit.plot(axis = fig.add_subplot(235),
#            title = "ASICS -> L1A (FIFOA)",
#            xlabel = "FIFO Occupancy",
#            ylabel = "Hits",
#            res = False, fit = False)
#    fig.add_subplot(235).set_yscale('log')
#    fig.add_subplot(235).text(0.4,0.9,(("L1_A FIFO reached %.1f %%" % \
#                                            (WC_L1_A_FIFO))),
#                                            fontsize=8,
#                                            verticalalignment='top',
#                                            horizontalalignment='left',
#                                            transform=fig.add_subplot(235).transAxes)
#    fit(out['L1']['logB'][:,0],CG['L1']['FIFO_L1b_depth'])
#    fit.plot(axis = fig.add_subplot(234),
#            title = "L1 OUTPUT (FIFOB)",
#            xlabel = "FIFO Occupancy",
#            ylabel = "Hits",
#            res = False, fit = False)
#    fig.add_subplot(234).set_yscale('log')
#    fig.add_subplot(234).text(0.4,0.9,(("L1_B FIFO reached %.1f %%" % \
#                                            (WC_L1_B_FIFO))),
#                                            fontsize=8,
#                                            verticalalignment='top',
#                                            horizontalalignment='left',
#                                            transform=fig.add_subplot(234).transAxes)
#    fit(latency,50)
#    fit.plot(axis = fig.add_subplot(233),
#            title = "Data Latency",
#            xlabel = "Latency in nanoseconds",
#            ylabel = "Hits",
#            res = False)
#    fig.add_subplot(233).text(0.4,0.9,(("WORST LATENCY = %d ns" % \
#                                            (max(latency)))),
#                                            fontsize=8,
#                                            verticalalignment='top',
#                                            horizontalalignment='left',
#                                            transform=fig.add_subplot(233).transAxes)
#    new_axis = fig.add_subplot(236)
#    x_data = fit.bin_centers
#    y_data = np.add.accumulate(fit.hist_fit)/np.max(np.add.accumulate(fit.hist_fit))
#    new_axis.plot(x_data,y_data)
#    new_axis.set_ylim((0.9,1.0))
#    new_axis.set_xlabel("Latency in nanoseconds")
#    new_axis.set_ylabel("Percentage of Recovered Data")
#    new_axis.text(0.05,0.9,(("LOST DATA PRODUCER -> CH           = %d\n" + \
#                             "LOST DATA CHANNELS -> OUTLINK  = %d\n" + \
#                             "LOST DATA OUTLINK -> L1                = %d\n" + \
#                             "LOST DATA L1A -> L1B                      = %d\n") % \
#                            (out['ASICS']['lost_producers'].sum(),
#                             out['ASICS']['lost_channels'].sum(),
#                             out['ASICS']['lost_outlink'].sum(),
#                             np.array(out['L1']['lostL1b']).sum())
#                            ),
#                            fontsize=8,
#                            verticalalignment='top',
#                            horizontalalignment='left',
#                            transform=new_axis.transAxes)
#
#    fig.tight_layout()

#plt.show()